In [1]:
import numpy as np
import sys
from PIL import ImageTk, Image, ImageFilter
import numpy as np
import matplotlib.pyplot
import warnings
import cv2
from PIL import Image, ImageEnhance
from scipy import ndimage
import os

from VirusDetectAlgo import *

'''
PATH = 'Mizuho_Feedback_Samples_0701'
reflection_time_para_map = {'FluA':5, 'FluB':5, 'Myco':15, 'RSV-hMPV':10, 'StrepA':5}
min_verify_concentration_para_map = {'FluA':'1600', 'FluB':'800', 'Myco':'800', 'RSV-hMPV':'0_200', 'StrepA':'1600'}
'''
#for factory dataset
PATH = 'Mizuho_only_golden_1023'
reflection_time_para_map = {'FluA':0, 'FluB':0, 'Myco':0, 'RSV-hMPV':0, 'StrepA':0}
min_verify_concentration_para_map = {'FluA':'2', 'FluB':'2', 'Myco':'2', 'RSV-hMPV':'2_2', 'StrepA':'2'}

FLAG_USE_GAMMA_ENHANCE = True
FLAG_USE_FILTER = True
smoothing_value = 3
FLAG_USE_PEAK_DETECTION = True
FLAG_USE_WHITE_BALANCE = True

FLAG_SHOW_FIGURE = False
FLAG_NUM_POSITIVE = 4

FLAG_STD_RATE = 2
RGB_Index_Range = 4
Stride_threshold_range = 30

SHOW_PICTURES = 5
Control_line_rate = 0.6

CURRENT_DISEASE = ['FluA', 'FluB', 'Myco', 'RSV-hMPV', 'StrepA']
disease_chosen = CURRENT_DISEASE[3]

PATH_neg = os.path.join(PATH, disease_chosen, '0')
tags = [disease_chosen+'_Neg', disease_chosen]

all_pos_files = []
c_folder = os.listdir(os.path.join(PATH, disease_chosen))
max_constraction = int(min_verify_concentration_para_map[disease_chosen])
for c in c_folder:
    if int(c)!=0 and int(c)<= max_constraction:
        PATH_pos = os.path.join(PATH, disease_chosen, c)
        if os.path.isdir(PATH_pos):
            files = os.listdir(PATH_pos)

            for f in files:
                if f.split('.')[-1]=='bmp':
                    all_pos_files.append(os.path.join(PATH_pos, f))

all_neg_files = []
if os.path.isdir(PATH_neg):
    files_neg = os.listdir(PATH_neg)        
    for f in files_neg:
        if f.split('.')[-1]=='bmp':
            all_neg_files.append(os.path.join(PATH_neg, f))

print('pos file len = ', len(all_pos_files))
print('neg file len = ', len(all_neg_files))

def openImg(path):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im



pos file len =  55
neg file len =  0


In [2]:
avg_control_threshold = []    
dif_value_array1 = []
dif_value_array2 = []
line_parameters = get_line_para_map(disease_chosen)

gt1 = []
pred1 = []
gt2 = []
pred2 = []

if len(all_neg_files)!=0:
    #calcul P/N first
    pn1, pn2 = calculateCutOffValue4Block(all_neg_files, line_parameters, FLAG_USE_GAMMA_ENHANCE, FLAG_USE_WHITE_BALANCE, FLAG_USE_FILTER, smoothing_value, FLAG_STD_RATE)
    
    line_parameters[1][4] = pn1
    if len(line_parameters)==3:
        line_parameters[2][4]= pn2
        
    #for confusion martix
    for neg_i in all_neg_files:
        im_n = Image.open(neg_i)
        control, test_result1, test_result2, test_result1_pred, test_result2_pred = algorithm3_with_slope_4block(im_n, line_parameters, FLAG_USE_GAMMA_ENHANCE, FLAG_USE_WHITE_BALANCE, FLAG_USE_FILTER, smoothing_value, FLAG_USE_PEAK_DETECTION, FLAG_SHOW_FIGURE, FLAG_NUM_POSITIVE)
        if control[0]==1:
            gt1.append(0)
            pred1.append(test_result1_pred)
            if test_result2!=None:
                gt2.append(0)
                pred2.append(test_result2_pred)
else:
    pn1 = line_parameters[1][4]
    if len(line_parameters)==3:
        pn2 = line_parameters[2][4]

print('pn1 = ', pn1, 'pn2 = ', pn2)

if len(all_pos_files)!=0:
    pos_control = []
    pos_list1 = []
    pos_list2 = []
    reflection_time = reflection_time_para_map[disease_chosen]
    for pos_i in all_pos_files:
        #find the ref time
        refect_time = pos_i.split('/')[-1].split('_')[-1].split('.')[0]
        if disease_chosen!='RSV-hMPV':
            time_index = refect_time.find('min')
            test_concentration1 = min_verify_concentration_para_map[disease_chosen]
            if time_index!=-1:
                c_ref_time_int = int(refect_time[:time_index])
                if c_ref_time_int>=reflection_time:
                    im_p = Image.open(pos_i)
                    control, test_result1, test_result2, test_result1_pred, test_result2_pred = algorithm3_with_slope_4block(im_p, line_parameters, FLAG_USE_GAMMA_ENHANCE, FLAG_USE_WHITE_BALANCE, FLAG_USE_FILTER, smoothing_value, FLAG_USE_PEAK_DETECTION, FLAG_SHOW_FIGURE, FLAG_NUM_POSITIVE)
                    pos_control.append(control[1])
                    if control[0]==1:
                        pos_list1.append(test_result1[0][1])
                        gt1.append(1)
                        pred1.append(int(test_result1[0][0]))
        else:    
            time_index = refect_time.find('min')
            if time_index!=-1:
                c_ref_time_int = int(refect_time[:time_index])
                if c_ref_time_int>=reflection_time:
                    im_p = Image.open(pos_i)
                    control, test_result1, test_result2, test_result1_pred, test_result2_pred = algorithm3_with_slope_4block(im_p, line_parameters, FLAG_USE_GAMMA_ENHANCE, FLAG_USE_WHITE_BALANCE, FLAG_USE_FILTER, smoothing_value, FLAG_USE_PEAK_DETECTION, FLAG_SHOW_FIGURE, FLAG_NUM_POSITIVE)
                    pos_control.append(control[1])
                    if control[0]==1:        
                        pos_list1.append(test_result1[0][1])
                        gt1.append(1)
                        pred1.append(test_result1_pred)              
                        if test_result2!=None:
                            pos_list2.append(test_result2[0][1])         
                            gt2.append(1)
                            pred2.append(test_result2_pred)

    avg_control = np.average(pos_control)
    avg_control_th = avg_control*Control_line_rate
    print('avg_control = ', avg_control, ' control threshold = ', avg_control_th)
    avg_control_threshold.append(avg_control_th)
            
    print('')
    avg_pos_value = np.average(pos_list1)
    diff_avg = avg_pos_value - pn1
    dif_value_array1.append(diff_avg)
    print('pos_list1 = ', pos_list1)
    print('avg_pos_value1 = ', avg_pos_value)
    print('diff_avg1 = ', diff_avg)
    print('gt1 = ', gt1)
    print('pred1 = ', pred1)
                
    accuracy, sensitivity, specificity, confusion = evaluate(pred1, gt1, tags)
    print('accuracy = ', accuracy, ' sensitivity = ', sensitivity, ' specificity ', specificity)
    print("rows are predicted classes, columns are actual classes")
    for predicted_index, predicted_tag in enumerate(tags):
        print(predicted_tag,end='')
        for actual_index, actual_tag in enumerate(tags):
            print("\t%d" % confusion[predicted_index, actual_index],end='')
        print()
    print()
    if test_result2!=[]:
        avg_pos_value = np.average(pos_list2)
        diff_avg = avg_pos_value - pn2
        dif_value_array2.append(diff_avg)
        print('pos_list2 = ', pos_list2)
        print('avg_pos_value2 = ', avg_pos_value)
        print('diff_avg2 = ', diff_avg)
        print('gt2 = ', gt2)
        print('pred2 = ', pred2)
        accuracy, sensitivity, specificity, confusion = evaluate(pred2, gt2, tags)
        print('accuracy = ', accuracy, ' sensitivity = ', sensitivity, ' specificity ', specificity)
        print("rows are predicted classes, columns are actual classes")
        for predicted_index, predicted_tag in enumerate(tags):
            print(predicted_tag,end='')
            for actual_index, actual_tag in enumerate(tags):
                print("\t%d" % confusion[predicted_index, actual_index],end='')
            print()
print('dif RGB testing result for control = ', avg_control_threshold)
print('dif RGB testing result for disease 1 = ', dif_value_array1)
if len(line_parameters)==3:
    print('dif RGB testing result for disease 2 = ', dif_value_array2)

pn1 =  10.211 pn2 =  7.261
avg_control =  97.5266  control threshold =  58.515962219238276

pos_list1 =  [6.769, 6.303, 7.354, 8.041, 5.749]
avg_pos_value1 =  6.8431997
diff_avg1 =  -3.3678002700805667
gt1 =  [1, 1, 1, 1, 1]
pred1 =  [0, 0, 0, 0, 0]
accuracy =  0.0  sensitivity =  2.3841857341722075e-08  specificity  1.0
rows are predicted classes, columns are actual classes
RSV-hMPV_Neg	0	5
RSV-hMPV	0	0

pos_list2 =  [8.015, 7.99, 8.436, 8.138, 7.636]
avg_pos_value2 =  8.043001
diff_avg2 =  0.7820011749267577
gt2 =  [1, 1, 1, 1, 1]
pred2 =  [1, 1, 1, 0, 1]
accuracy =  0.8  sensitivity =  0.8000000047683715  specificity  1.0
rows are predicted classes, columns are actual classes
RSV-hMPV_Neg	0	1
RSV-hMPV	0	4
dif RGB testing result for control =  [58.515962219238276]
dif RGB testing result for disease 1 =  [-3.3678002700805667]
dif RGB testing result for disease 2 =  [0.7820011749267577]


In [2]:
#show all result
for index, a in enumerate(all_neg_files):
    im = openImg(a)
    print('data name = ', a)
    #show_plot(im)
    line_parameters = get_line_para_map(disease_chosen)
    control, test_result1, test_result2, test_result1_pred, test_result2_pred = algorithm3_with_slope_4block(im, line_parameters, FLAG_USE_GAMMA_ENHANCE, FLAG_USE_WHITE_BALANCE, FLAG_USE_FILTER, smoothing_value, FLAG_USE_PEAK_DETECTION, FLAG_SHOW_FIGURE, FLAG_NUM_POSITIVE)
    
    if control[0]!=0:
        for t1 in test_result1:
            print('Result = ', t1[:2], "test_result1_pred = ", test_result1_pred) 
    
for index, a in enumerate(all_pos_files):
    im = openImg(a)
    print('data name = ', a)
    #show_plot(im)
    line_parameters = get_line_para_map(disease_chosen)
    control, test_result1, test_result2, test_result1_pred, test_result2_pred = algorithm3_with_slope_4block(im, line_parameters, FLAG_USE_GAMMA_ENHANCE, FLAG_USE_WHITE_BALANCE, FLAG_USE_FILTER, smoothing_value, FLAG_USE_PEAK_DETECTION, FLAG_SHOW_FIGURE, FLAG_NUM_POSITIVE)
    
    if control[0]!=0:
        for t1 in test_result1:
            print('Result = ', t1[:2], "test_result1_pred = ", test_result1_pred) 

data name =  Mizuho_Feedback_Samples_0701/StrepA/0/image_bitmap_20190524153959_StrepA_4min.bmp
Result =  [1, 6.09] test_result1_pred =  0
Result =  [1, 7.701] test_result1_pred =  0
Result =  [1, 6.851] test_result1_pred =  0
Result =  [0, 5.358] test_result1_pred =  0
data name =  Mizuho_Feedback_Samples_0701/StrepA/0/image_bitmap_20190524160240_StrepA_5min.bmp
Result =  [1, 6.194] test_result1_pred =  1
Result =  [1, 5.836] test_result1_pred =  1
Result =  [1, 6.328] test_result1_pred =  1
Result =  [1, 5.701] test_result1_pred =  1
data name =  Mizuho_Feedback_Samples_0701/StrepA/0/image_bitmap_20190524155206_StrepA_5min.bmp
Result =  [1, 5.567] test_result1_pred =  0
Result =  [1, 7.627] test_result1_pred =  0
Result =  [1, 7.701] test_result1_pred =  0
Result =  [0, 4.806] test_result1_pred =  0
data name =  Mizuho_Feedback_Samples_0701/StrepA/0/image_bitmap_20190524161332_StrepA_5min.bmp
Result =  [0, 5.343] test_result1_pred =  0
Result =  [1, 7.224] test_result1_pred =  0
Resul

Result =  [1, 7.94] test_result1_pred =  0
Result =  [1, 7.448] test_result1_pred =  0
Result =  [0, 5.433] test_result1_pred =  0
Result =  [1, 8.06] test_result1_pred =  0
data name =  Mizuho_Feedback_Samples_0701/StrepA/1600/image_bitmap_20190524145953_StrepA_3min.bmp
Result =  [1, 7.373] test_result1_pred =  1
Result =  [1, 8.388] test_result1_pred =  1
Result =  [1, 11.657] test_result1_pred =  1
Result =  [1, 9.552] test_result1_pred =  1
data name =  Mizuho_Feedback_Samples_0701/StrepA/1600/image_bitmap_20190524152532_StrepA_1min.bmp
data name =  Mizuho_Feedback_Samples_0701/StrepA/1600/image_bitmap_20190524144859_StrepA_2min.bmp
Result =  [1, 10.284] test_result1_pred =  1
Result =  [1, 9.015] test_result1_pred =  1
Result =  [1, 9.746] test_result1_pred =  1
Result =  [1, 8.045] test_result1_pred =  1
data name =  Mizuho_Feedback_Samples_0701/StrepA/1600/image_bitmap_20190524144959_StrepA_3min.bmp
Result =  [1, 9.448] test_result1_pred =  1
Result =  [1, 8.672] test_result1_pr

In [3]:
exit()